In [1]:
## Workout Analytics
#### 01. Aquisition with Crawling
#### 02. Transformation with Cleansing, Denoising, Imputation and Normalization(using Exercise-Dictionary)
#### 03. Exploring Basically

In [ ]:
import ssl
from urllib.request import Request, urlopen
from functools import reduce
from itertools import count

try:
    ssl._create_default_https_context = ssl._create_unverified_context

    resfetch = []
    for idx in count(start=1):
        l = len(resfetch)
        resfetch[l:] = list(filter(
            lambda s: s.startswith('#'),
            reduce(
                lambda param, func: func(param),
                [lambda raw: [] if raw is None else raw.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
                urlopen(Request('https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/datasets/Week0%02d.md' % idx)).read().decode('utf-8', errors='replace')
            )
        ))

        if len(resfetch) == l:
            break

    print(f'Aquisition Raw Data Count: {len(resfetch)}')
except Exception as err:
    (lambda e: None)(err)

In [ ]:
from pathlib import Path
import os
import pickle

dictfile = os.path.join(Path(os.getcwd()).parent, 'datasets', 'pickles', 'exercise-dict.pkl')
with open(dictfile, 'rb') as file:
    exercisedict = pickle.load(file)

print(f'Loding Exercise Dictionary Pickle[{dictfile}]')

In [ ]:
import re
from datetime import datetime

restransform = []
exs_dict = exercisedict['exercises']
eqptdials_dict = exercisedict['equipment-dialects']

exercise_rawdata_count = 0
date_rawdata_count = 0

for res in resfetch:
    if res.startswith('####'):
        exercise_rawdata_count = exercise_rawdata_count + 1

        # 1. split exercise name & record
        exname, exrecord = (s.strip() for s in res.replace('####', '').strip().split(':'))

        # 2. normalize exercise name with equipment
        exname = re.sub(r'[\d+\\.]', '', exname).strip()
        pttrmatch = re.compile(r'([a-zA-Z\s\',()\-]+)\s*\[([a-zA-Z\s\',]+)\]').match(exname)
        exname, equipment = (exname, None) if pttrmatch is None else pttrmatch.groups()

        # 2-1. name
        exname = exname.strip().title()

        for exkey in exs_dict:
            exdials = exs_dict[exkey]['dialects'] if 'dialects' in exs_dict[exkey] else []
            if exname in exdials:
                exname = exkey
                break

        # 2-2. equipment
        if exname in exs_dict and 'default-equipment' in exs_dict[exname]:
            if equipment or \
                    ('equipment-dialects' in exs_dict[exname] and equipment in exs_dict[exname]['equipment-dialects']):
                equipment = exs_dict[exname]['default-equipment']

        if equipment:
            equipment = equipment.strip().title()

        for eqptdials_key in eqptdials_dict:
            if equipment in eqptdials_dict[eqptdials_key]:
                equipment = eqptdials_key
                break

        # 3. Display Name Compounded
        exname = exname + (f'[{equipment}]' if equipment else '')

        # 4.
        # sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()

        restransform[len(restransform):] = [(date, exname, *re.split(r'\s+', exrecord.strip()))]
    elif res.startswith('##'):
        date_rawdata_count = date_rawdata_count + 1
        date = datetime.strptime(res.replace('##', '').strip(), '%Y/%m/%d')

print(f'Raw Data Count(Exercises, Days): ({exercise_rawdata_count}, {date_rawdata_count})')
print(f'Exercise Transform Data(Tuple) Count: {len(restransform)}')

In [ ]:
# 03. Exploring1: How Exercise Days?
days = list(set([t[0] for t in restransform]))
print(f'Exercise Days: {len(days)}')

In [ ]:
# 03. Exploring2: Which Exercises?
exs = [t[1] for t in restransform]
dataset_exs = [(ex, exs.count(ex)) for ex in list(set(exs))]

sorted_by_ex = sorted(dataset_exs, key=lambda t: t[0])
for d in sorted_by_ex:
    print(d)

In [ ]:
# 03. Exploring3: and How Often?
sorted_by_count = sorted(dataset_exs, key=lambda t: t[1], reverse=True)
for d in sorted_by_count:
     print(d)